In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv("openaq.csv", sep=";")

In [3]:
coordinates = data['Coordinates'].str.split(',', expand=True)
data['Latitude'] = coordinates[0].astype(float)
data['Longitude'] = coordinates[1].astype(float)

data.drop('Coordinates', axis=1, inplace=True)

In [4]:
new_order = ['Last Updated', 'Value', 'Pollutant', 'Longitude', 'Latitude', 'City']
data = data[new_order]

In [5]:
data.to_csv('reorganized_openaq.csv', index=False)

In [6]:
data = pd.read_csv("reorganized_openaq.csv", sep=",")
data

,Last Updated,Value,Pollutant,Longitude,Latitude,City
0,2024-04-05T10:00:00+02:00,9.550000,NO2,53.703790,23.652199,Abu Dhabi
1,2024-04-05T10:00:00+02:00,57.000000,PM10,53.606390,23.095690,Liwa
2,2024-04-05T10:00:00+02:00,3.380000,SO2,54.502800,24.347150,Abu Dhabi
3,2024-04-05T10:00:00+02:00,53.940000,NO2,54.502800,24.347150,Abu Dhabi
4,2024-04-05T10:00:00+02:00,50.680000,NO2,54.586100,24.286970,Abu Dhabi
...,...,...,...,...,...,...
24704,2024-04-05T09:00:00+02:00,22.350000,PM10,4.329430,51.914883,NaN
24705,2024-04-05T09:00:00+02:00,4.390000,NO2,6.042399,51.119194,POSTERHOLT
24706,2024-01-12T05:00:00+01:00,42.000000,O3,4.339710,51.348795,Antwerpen
24707,2024-04-05T10:00:00+02:00,2.479444,PM10,10.661762,59.433471,Moss


In [7]:
data['Last Updated'] = pd.to_datetime(data['Last Updated'], utc=True).dt.tz_convert(None)

In [8]:
data.rename(columns={'Last Updated': 'Date'}, inplace=True)

In [9]:
# Extraire les composantes de la date
data['Jour'] = data['Date'].dt.day
data['Mois'] = data['Date'].dt.month
data['Année'] = data['Date'].dt.year
data['Heure'] = data['Date'].dt.hour
data['Minutes'] = data['Date'].dt.minute

In [10]:
pollutant_dummies = pd.get_dummies(data['Pollutant'], prefix='Pollutant')

print(pollutant_dummies.head())

   Pollutant_BC  Pollutant_CO  Pollutant_NO  Pollutant_NO2  Pollutant_NOX  \
0         False         False         False           True          False   
1         False         False         False          False          False   
2         False         False         False          False          False   
3         False         False         False           True          False   
4         False         False         False           True          False   

   Pollutant_O3  Pollutant_PM1  Pollutant_PM10  Pollutant_PM2.5  \
0         False          False           False            False   
1         False          False            True            False   
2         False          False           False            False   
3         False          False           False            False   
4         False          False           False            False   

   Pollutant_RELATIVEHUMIDITY  Pollutant_SO2  Pollutant_TEMPERATURE  \
0                       False          False                  F

In [11]:
data = pd.concat([data, pollutant_dummies], axis=1)

data.drop('Pollutant', axis=1, inplace=True)

print(data.head())

                 Date  Value  Longitude   Latitude       City  Jour  Mois  \
0 2024-04-05 08:00:00   9.55   53.70379  23.652199  Abu Dhabi     5     4   
1 2024-04-05 08:00:00  57.00   53.60639  23.095690       Liwa     5     4   
2 2024-04-05 08:00:00   3.38   54.50280  24.347150  Abu Dhabi     5     4   
3 2024-04-05 08:00:00  53.94   54.50280  24.347150  Abu Dhabi     5     4   
4 2024-04-05 08:00:00  50.68   54.58610  24.286970  Abu Dhabi     5     4   

   Année  Heure  Minutes  ...  Pollutant_NO2  Pollutant_NOX  Pollutant_O3  \
0   2024      8        0  ...           True          False         False   
1   2024      8        0  ...          False          False         False   
2   2024      8        0  ...          False          False         False   
3   2024      8        0  ...           True          False         False   
4   2024      8        0  ...           True          False         False   

   Pollutant_PM1  Pollutant_PM10  Pollutant_PM2.5  Pollutant_RELATIVEHUMID

In [12]:
columns_to_convert = ['Pollutant_BC', 'Pollutant_CO', 'Pollutant_NO', 'Pollutant_NO2', 'Pollutant_NOX', 'Pollutant_O3', 'Pollutant_PM1', 'Pollutant_PM10', 'Pollutant_PM2.5', 'Pollutant_SO2']

for col in columns_to_convert:
    data[col] = data[col].replace({True: 1, False: 0})

print(data)

                     Date      Value  Longitude   Latitude        City  Jour  \
0     2024-04-05 08:00:00   9.550000  53.703790  23.652199   Abu Dhabi     5   
1     2024-04-05 08:00:00  57.000000  53.606390  23.095690        Liwa     5   
2     2024-04-05 08:00:00   3.380000  54.502800  24.347150   Abu Dhabi     5   
3     2024-04-05 08:00:00  53.940000  54.502800  24.347150   Abu Dhabi     5   
4     2024-04-05 08:00:00  50.680000  54.586100  24.286970   Abu Dhabi     5   
...                   ...        ...        ...        ...         ...   ...   
24704 2024-04-05 07:00:00  22.350000   4.329430  51.914883         NaN     5   
24705 2024-04-05 07:00:00   4.390000   6.042399  51.119194  POSTERHOLT     5   
24706 2024-01-12 04:00:00  42.000000   4.339710  51.348795   Antwerpen    12   
24707 2024-04-05 08:00:00   2.479444  10.661762  59.433471        Moss     5   
24708 2024-04-05 08:00:00  91.599258  10.801863  59.928454        Oslo     5   

       Mois  Année  Heure  Minutes  ...

C:\Users\diogo\AppData\Local\Temp\ipykernel_12840\1566856976.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace({True: 1, False: 0})


In [13]:
data.reset_index(drop=True, inplace=True)
data

,Date,Value,Longitude,Latitude,City,Jour,Mois,Année,Heure,Minutes,...,Pollutant_NO2,Pollutant_NOX,Pollutant_O3,Pollutant_PM1,Pollutant_PM10,Pollutant_PM2.5,Pollutant_RELATIVEHUMIDITY,Pollutant_SO2,Pollutant_TEMPERATURE,Pollutant_UM003
0,2024-04-05 08:00:00,9.550000,53.703790,23.652199,Abu Dhabi,5,4,2024,8,0,...,1,0,0,0,0,0,False,0,False,False
1,2024-04-05 08:00:00,57.000000,53.606390,23.095690,Liwa,5,4,2024,8,0,...,0,0,0,0,1,0,False,0,False,False
2,2024-04-05 08:00:00,3.380000,54.502800,24.347150,Abu Dhabi,5,4,2024,8,0,...,0,0,0,0,0,0,False,1,False,False
3,2024-04-05 08:00:00,53.940000,54.502800,24.347150,Abu Dhabi,5,4,2024,8,0,...,1,0,0,0,0,0,False,0,False,False
4,2024-04-05 08:00:00,50.680000,54.586100,24.286970,Abu Dhabi,5,4,2024,8,0,...,1,0,0,0,0,0,False,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24704,2024-04-05 07:00:00,22.350000,4.329430,51.914883,NaN,5,4,2024,7,0,...,0,0,0,0,1,0,False,0,False,False
24705,2024-04-05 07:00:00,4.390000,6.042399,51.119194,POSTERHOLT,5,4,2024,7,0,...,1,0,0,0,0,0,False,0,False,False
24706,2024-01-12 04:00:00,42.000000,4.339710,51.348795,Antwerpen,12,1,2024,4,0,...,0,0,1,0,0,0,False,0,False,False
24707,2024-04-05 08:00:00,2.479444,10.661762,59.433471,Moss,5,4,2024,8,0,...,0,0,0,0,1,0,False,0,False,False


In [15]:
data.to_csv('AQI.csv', index=False)

In [16]:
data['Date'] = pd.to_datetime(data['Date'])
data['Jour'] = data['Date'].dt.day
data['Mois'] = data['Date'].dt.month
data['Année'] = data['Date'].dt.year
data['Heure'] = data['Date'].dt.hour

# Agrégation des valeurs par Date, Heure et Ville
aggregated_data = data.groupby(['Date', 'City']).agg({
    'Value': 'mean',
    'Longitude': 'mean',
    'Latitude': 'mean',
}).reset_index()

In [23]:
aggregated_data.to_csv('aggregated_data.csv', index=False)

In [20]:
aggregated_data['Jour'] = aggregated_data['Date'].dt.day
aggregated_data['Mois'] = aggregated_data['Date'].dt.month
aggregated_data['Année'] = aggregated_data['Date'].dt.year
aggregated_data['Heure'] = aggregated_data['Date'].dt.hour
aggregated_data['Minutes'] = aggregated_data['Date'].dt.minute

In [21]:
aggregated_data

,Date,City,Value,Longitude,Latitude,Jour,Mois,Année,Heure,Minutes
0,2024-01-01 00:00:00,Madrid,4.636364,-3.705036,40.423759,1,1,2024,0,0
1,2024-01-01 00:00:00,Seine-Maritime,28.066667,0.267360,49.520565,1,1,2024,0,0
2,2024-01-01 02:00:00,Kielce,2.000000,20.642858,50.886014,1,1,2024,2,0
3,2024-01-01 06:00:00,Durango,15.534024,-104.645028,24.034472,1,1,2024,6,0
4,2024-01-01 22:00:00,Córdoba,1.555556,-4.911580,38.128000,1,1,2024,22,0
...,...,...,...,...,...,...,...,...,...,...
4669,2024-04-05 08:30:00,Chennai,5.000000,80.278470,13.087840,5,4,2024,8,30
4670,2024-04-05 08:30:00,Delhi,-999.000000,77.224450,28.635760,5,4,2024,8,30
4671,2024-04-05 08:30:00,Hyderabad,34.000000,78.456360,17.384050,5,4,2024,8,30
4672,2024-04-05 08:30:00,Kolkata,100.000000,88.363040,22.562630,5,4,2024,8,30
